In [128]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
pd.options.mode.chained_assignment = None  # default='warn'

In [129]:
# Other imports

# Get Data


In [131]:
utilData = pd.read_csv('./Util_Run1.csv')
utilData['memLoad'] = utilData['memUse']*100/utilData['memTot']
utilData['memScore'] = utilData['memLoad']*utilData['memfreq']/825000000
utilData['memF'] = utilData['memfreq']*100/825000000
utilData['cpuScore'] = utilData['cpuload']*utilData['cpufreq']/1400000
utilData['cpuF'] = utilData['cpufreq']*100/1000000

In [132]:
# Some data modification
utilData

,memUse,memTot,memfreq,cpuid,cpuload,cpufreq,memLoad,memScore,memF,cpuScore,cpuF
0,604872,4612865,165000000,0,31,600000,13.112718,2.622544,20.000000,13.285714,60.0
1,675,28712,275000000,0,4,200000,2.350933,0.783644,33.333333,0.571429,20.0
2,1191888,12624777,413000000,0,8,200000,9.440864,4.726154,50.060606,1.142857,20.0
3,773492,8196989,275000000,0,9,300000,9.436294,3.145431,33.333333,1.928571,30.0
4,1144,57890,206000000,0,7,300000,1.976162,0.493442,24.969697,1.500000,30.0
...,...,...,...,...,...,...,...,...,...,...,...
2669,807320,11546069,413000000,0,4,200000,6.992163,3.500319,50.060606,0.571429,20.0
2670,195717,2817255,275000000,0,7,300000,6.947081,2.315694,33.333333,1.500000,30.0
2671,1344,50569,275000000,0,4,300000,2.657755,0.885918,33.333333,0.857143,30.0
2672,1344,50569,275000000,0,5,300000,2.657755,0.885918,33.333333,1.071429,30.0


In [133]:
# utilData.shift(1)
utilData.shape[0]

2674

# Fuzzy AR-MA


In [156]:
# def ART(p,endog,exog=None,data_weight=None):
    
#     df = pd.DataFrame()
#     df['endog'] = endog
#     if exog!= None:
#         df['exog'] = exog
    
    
df = pd.DataFrame()
# df['Weight'] = np.ones(utilData.shape[0])
df['Value'] = utilData['cpuScore'].diff(1)
df['Weight'] = np.ones(utilData.shape[0])
df['exog'] = utilData['memLoad'].shift(1)
# df.iloc[:,1]
df

,Value,Weight,exog
0,NaN,1.0,NaN
1,-12.714286,1.0,13.112718
2,0.571429,1.0,2.350933
3,0.785714,1.0,9.440864
4,-0.428571,1.0,9.436294
...,...,...,...
2669,-0.714286,1.0,6.992163
2670,0.928571,1.0,6.992163
2671,-0.642857,1.0,6.947081
2672,0.214286,1.0,2.657755


In [217]:
from statsmodels.tools.eval_measures import rmspe
def FuzzyARX(p,df):
    df_temp = pd.DataFrame()
    df_temp['Value'] = df['Value']

    #Generating the lagged p terms
    for i in range(1,p+1):
        df_temp['Shifted_values_%d' % i ] = df_temp['Value'].shift(i)    
    df_temp['exog'] = df['exog']
    df_temp['Weight'] = df['Weight']
    train_size = (int)(0.8 * df_temp.shape[0])
    #Breaking data set into test and training
    df_train = pd.DataFrame(df_temp[0:train_size])
    df_test = pd.DataFrame(df_temp[train_size:df.shape[0]])
    df_train_2 = df_train.dropna()
    X_train = df_train_2.iloc[:,1:p+2].values.reshape(-1,p+1)
    #Y contains the value,it is the first column
    y_train = df_train_2.iloc[:,0].values.reshape(-1,1)
    print(y_train.shape[0])
    sample_weight = df_train_2['Weight']
    #Running linear regression to generate the coefficents of lagged terms
    from sklearn.linear_model import LinearRegression
    lr = LinearRegression()
    lr.fit(X_train,y_train,sample_weight)
    theta  = lr.coef_.T
    intercept = lr.intercept_
    df_train_2['Predicted_Values'] = X_train.dot(lr.coef_.T) + lr.intercept_
    # df_train_2[['Value','Predicted_Values']].plot()

    X_test = df_test.iloc[:,1:p+2].values.reshape(-1,p+1)
    df_test['Predicted_Values'] = X_test.dot(lr.coef_.T) + lr.intercept_
    # df_test[['Value','Predicted_Values']].plot()
#     RMSE = np.sqrt(mean_squared_error(df_test['Value'], df_test['Predicted_Values']))
    RMSE = rmspe(df_test['Value'],df_test['Predicted_Values'])

    print("The RMSE is :", RMSE,", Value of p : ",p)
    return [df_train_2,df_test,theta,intercept,RMSE]

In [218]:
val = FuzzyARX(4,df)

2134
The RMSE is : 22.210738303310432 , Value of p :  4
